<a href="https://colab.research.google.com/github/rsanch12/ECGR-4105-FALL_2025/blob/main/Homework%205/Problem_2_HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# use seaborn plotting defaults
import seaborn as sns; sns.set()

In [2]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/HamedTabkhi/Intro-to-ML/refs/heads/main/Dataset/Housing.csv")
display(df.head())

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [3]:
categorical_cols = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
display(df.head())

,price,area,bedrooms,bathrooms,stories,parking,mainroad_yes,guestroom_yes,basement_yes,hotwaterheating_yes,airconditioning_yes,prefarea_yes,furnishingstatus_semi-furnished,furnishingstatus_unfurnished
0,13300000,7420,4,2,3,2,True,False,False,False,True,True,False,False
1,12250000,8960,4,4,4,3,True,False,False,False,True,False,False,False
2,12250000,9960,3,2,2,2,True,False,True,False,False,True,True,False
3,12215000,7500,4,2,2,3,True,False,True,False,True,True,False,False
4,11410000,7420,4,1,2,2,True,True,True,False,True,False,False,False


## Part A


In [4]:
from sklearn.model_selection import train_test_split

features = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking']
target = 'price'

X = df[features]
y = df[target]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
num_features = X_train.shape[1]
weights = np.zeros(num_features)
bias = 0

## Part B


In [6]:
def forward_pass(X, weights, bias):
  """Calculates the predicted target values."""
  return np.dot(X, weights) + bias

def mse_loss(y_pred, y_true):
  """Calculates the Mean Squared Error loss."""
  return np.mean((y_pred - y_true)**2)

def train_model(X_train, y_train, X_val, y_val, weights, bias, learning_rate, epochs):
  """Trains the linear regression model."""
  train_losses = []
  val_losses = []

  for epoch in range(epochs):
    # Forward pass
    y_train_pred = forward_pass(X_train, weights, bias)

    # Calculate training loss
    loss = mse_loss(y_train_pred, y_train)
    train_losses.append(loss)

    # Backpropagation
    d_weights = (2/len(X_train)) * np.dot(X_train.T, (y_train_pred - y_train))
    d_bias = (2/len(X_train)) * np.sum(y_train_pred - y_train)

    # Update parameters
    weights -= learning_rate * d_weights
    bias -= learning_rate * d_bias

    # Evaluate on validation set periodically
    if (epoch + 1) % 500 == 0:
      y_val_pred = forward_pass(X_val, weights, bias)
      val_loss = mse_loss(y_val_pred, y_val)
      val_losses.append(val_loss)
      print(f"Epoch {epoch+1}/{epochs}, Train Loss: {loss:.4f}, Validation Loss: {val_loss:.4f}")

  return weights, bias, train_losses, val_losses

# Example usage with one learning rate (will be iterated later)
learning_rate = 0.01
epochs = 5000
weights, bias, train_losses, val_losses = train_model(X_train, y_train, X_val, y_val, weights, bias, learning_rate, epochs)

/tmp/ipython-input-2938189509.py:27: RuntimeWarning: invalid value encountered in subtract
  weights -= learning_rate * d_weights


Epoch 500/5000, Train Loss: nan, Validation Loss: nan
Epoch 1000/5000, Train Loss: nan, Validation Loss: nan
Epoch 1500/5000, Train Loss: nan, Validation Loss: nan
Epoch 2000/5000, Train Loss: nan, Validation Loss: nan
Epoch 2500/5000, Train Loss: nan, Validation Loss: nan
Epoch 3000/5000, Train Loss: nan, Validation Loss: nan
Epoch 3500/5000, Train Loss: nan, Validation Loss: nan
Epoch 4000/5000, Train Loss: nan, Validation Loss: nan
Epoch 4500/5000, Train Loss: nan, Validation Loss: nan
Epoch 5000/5000, Train Loss: nan, Validation Loss: nan


**Reasoning**:
The training resulted in NaN values for loss, indicating potential issues with the learning rate or data scaling. Scale the features and target variable before retraining.



In [7]:
from sklearn.preprocessing import StandardScaler

scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_val_scaled = scaler_X.transform(X_val)

scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).flatten()
y_val_scaled = scaler_y.transform(y_val.values.reshape(-1, 1)).flatten()

# Re-initialize weights and bias for the scaled data
weights_scaled = np.zeros(num_features)
bias_scaled = 0

# Train the model with scaled data
learning_rate = 0.01
epochs = 5000
weights_scaled, bias_scaled, train_losses_scaled, val_losses_scaled = train_model(X_train_scaled, y_train_scaled, X_val_scaled, y_val_scaled, weights_scaled, bias_scaled, learning_rate, epochs)

Epoch 500/5000, Train Loss: 0.4378, Validation Loss: 0.7436
Epoch 1000/5000, Train Loss: 0.4378, Validation Loss: 0.7436
Epoch 1500/5000, Train Loss: 0.4378, Validation Loss: 0.7436
Epoch 2000/5000, Train Loss: 0.4378, Validation Loss: 0.7436
Epoch 2500/5000, Train Loss: 0.4378, Validation Loss: 0.7436
Epoch 3000/5000, Train Loss: 0.4378, Validation Loss: 0.7436
Epoch 3500/5000, Train Loss: 0.4378, Validation Loss: 0.7436
Epoch 4000/5000, Train Loss: 0.4378, Validation Loss: 0.7436
Epoch 4500/5000, Train Loss: 0.4378, Validation Loss: 0.7436
Epoch 5000/5000, Train Loss: 0.4378, Validation Loss: 0.7436


In [8]:
def train_model(X_train, y_train, X_val, y_val, weights, bias, learning_rate, epochs):
    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        # Forward pass
        y_pred = np.dot(X_train, weights) + bias

        # Loss calculation (Mean Squared Error)
        train_loss = np.mean((y_pred - y_train)**2)
        train_losses.append(train_loss)

        # Backpropagation
        d_weights = (2/len(X_train)) * np.dot(X_train.T, (y_pred - y_train))
        d_bias = (2/len(X_train)) * np.sum(y_pred - y_train)

        # Update weights and bias
        weights -= learning_rate * d_weights
        bias -= learning_rate * d_bias

        # Evaluate on validation set
        y_val_pred = np.dot(X_val, weights) + bias
        val_loss = np.mean((y_val_pred - y_val)**2)
        val_losses.append(val_loss)

        # Report loss every 500 epochs
        if (epoch + 1) % 500 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Learning Rate: {learning_rate}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

    return weights, bias, train_losses, val_losses

learning_rates = [0.1, 0.01, 0.001, 0.0001]
losses_by_lr = {}

for lr in learning_rates:
    print(f"\nTraining with Learning Rate: {lr}")
    # Re-initialize weights and bias for each learning rate
    weights_scaled = np.zeros(num_features)
    bias_scaled = 0

    weights_scaled_trained, bias_scaled_trained, train_losses_scaled, val_losses_scaled = train_model(X_train_scaled, y_train_scaled, X_val_scaled, y_val_scaled, weights_scaled, bias_scaled, lr, epochs)
    losses_by_lr[lr] = {'train_losses': train_losses_scaled, 'val_losses': val_losses_scaled}



Training with Learning Rate: 0.1
Epoch 500/5000, Learning Rate: 0.1, Train Loss: 0.4378, Validation Loss: 0.7436
Epoch 1000/5000, Learning Rate: 0.1, Train Loss: 0.4378, Validation Loss: 0.7436
Epoch 1500/5000, Learning Rate: 0.1, Train Loss: 0.4378, Validation Loss: 0.7436
Epoch 2000/5000, Learning Rate: 0.1, Train Loss: 0.4378, Validation Loss: 0.7436
Epoch 2500/5000, Learning Rate: 0.1, Train Loss: 0.4378, Validation Loss: 0.7436
Epoch 3000/5000, Learning Rate: 0.1, Train Loss: 0.4378, Validation Loss: 0.7436
Epoch 3500/5000, Learning Rate: 0.1, Train Loss: 0.4378, Validation Loss: 0.7436
Epoch 4000/5000, Learning Rate: 0.1, Train Loss: 0.4378, Validation Loss: 0.7436
Epoch 4500/5000, Learning Rate: 0.1, Train Loss: 0.4378, Validation Loss: 0.7436
Epoch 5000/5000, Learning Rate: 0.1, Train Loss: 0.4378, Validation Loss: 0.7436

Training with Learning Rate: 0.01
Epoch 500/5000, Learning Rate: 0.01, Train Loss: 0.4378, Validation Loss: 0.7436
Epoch 1000/5000, Learning Rate: 0.01, Tra

In [9]:
best_lr = None
min_val_loss = float('inf')

for lr, losses in losses_by_lr.items():
    final_val_loss = losses['val_losses'][-1]
    if final_val_loss < min_val_loss:
        min_val_loss = final_val_loss
        best_lr = lr

print(f"The best learning rate is: {best_lr} with a final validation loss of: {min_val_loss:.4f}")

The best learning rate is: 0.1 with a final validation loss of: 0.7436
